In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
STORM_NUMBER = 16
TITLE = "1-2 April 2022"
SUBTITLE = "8 AM 3 April 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 4, 2, 6)
ets = utc(2022, 4, 3, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [4]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [5]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        geo = gpd.points_from_xy(x=[SETPOINT_LOCS[sp][0]], y=[SETPOINT_LOCS[sp][1]], crs=df.crs)
        df.loc[10000 + sp] = {"geo": geo[0], "val": val, "used_for_analysis": True,
                             "plotme": True}
    df["geo"] = gpd.GeoSeries(df["geo"], crs=df.crs)

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [9]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = ['WMTI4']
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='220403.png')
    mp.close()

main()

     state  wfo  val      lon      lat                             geo  \
0       IL  DVN  5.0 -89.5700  42.3200  POINT (854878.277 -243448.595)   
1       IL  DVN  4.9 -89.4100  42.4600  POINT (865939.819 -226293.027)   
2       IL  LOT  4.5 -89.0800  42.4500  POINT (892871.691 -223848.087)   
3       IA  DVN  4.5 -90.7300  42.4100  POINT (759234.612 -244937.769)   
4       IL  DVN  4.5 -89.6300  42.2900  POINT (850404.608 -247389.753)   
...    ...  ...  ...      ...      ...                             ...   
2021    KS  ICT  0.0 -98.1136  37.6361  POINT (166434.612 -816389.286)   
2022    KS  ICT  0.0 -97.4430  37.6550  POINT (225521.246 -812722.487)   
2023    KS  TOP  0.0 -96.3500  38.6400  POINT (317375.421 -699884.398)   
2024    KS  TOP  0.0 -97.1275  39.3738  POINT (247146.302 -621079.297)   
2025    KS  TOP  0.0 -97.4600  39.8000  POINT (217171.648 -574687.826)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0                  True      0    True    LSR    